In [1]:
import numpy as np
from scipy.stats import norm
import time # To compare execution times

# --- 1. Black-Scholes Model ---
def black_scholes(S0, K, T, r, sigma, option_type='call'):
    """
    Calculates the Black-Scholes option price for a European option.

    Parameters:
    S0 (float): Current stock price
    K (float): Option strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate (annualized, e.g., 0.05 for 5%)
    sigma (float): Volatility of the underlying stock (annualized, e.g., 0.2 for 20%)
    option_type (str): 'call' or 'put'

    Returns:
    float: Option price
    """
    if T == 0: # If time to expiration is 0, return intrinsic value
        if option_type == 'call':
            return max(0, S0 - K)
        elif option_type == 'put':
            return max(0, K - S0)
        else:
            raise ValueError("Invalid option type. Must be 'call' or 'put'.")

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Must be 'call' or 'put'.")
    return price

# --- 2. Binomial Model (Cox-Ross-Rubinstein) ---
def binomial_model_crr(S0, K, T, r, sigma, N, option_type='call'):
    """
    Calculates the European option price using the Cox-Ross-Rubinstein Binomial Model.

    Parameters:
    S0 (float): Current stock price
    K (float): Option strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate (annualized)
    sigma (float): Volatility of the underlying stock (annualized)
    N (int): Number of time steps in the binomial tree
    option_type (str): 'call' or 'put'

    Returns:
    float: Option price
    """
    if T == 0: # If time to expiration is 0, return intrinsic value
        if option_type == 'call':
            return max(0, S0 - K)
        elif option_type == 'put':
            return max(0, K - S0)
        else:
            raise ValueError("Invalid option type. Must be 'call' or 'put'.")
            
    # Calculate parameters
    dt = T / N  # Length of each time step
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor (d = np.exp(-sigma * np.sqrt(dt)))
    
    # Risk-neutral probability
    # Check for arbitrage opportunity (if q is not between 0 and 1, model might be misspecified for high N and specific r, sigma)
    # exp(r*dt) must be between d and u.
    if not (d < np.exp(r * dt) < u):
        # This can happen if dt is too large relative to r and sigma, or N is too small.
        # Or if sigma is very small.
        # For CRR, if sigma is 0, u=d=1, q is undefined. Handle this.
        if sigma == 0: # Deterministic case
            S_T_deterministic = S0 * np.exp(r * T)
            if option_type == 'call':
                return max(0, S_T_deterministic - K) * np.exp(-r * T)
            else:
                return max(0, K - S_T_deterministic) * np.exp(-r * T)
        # print(f"Warning: Risk-neutral probability might be outside (0,1). Check parameters. u={u}, d={d}, exp(r*dt)={np.exp(r*dt)}")


    q = (np.exp(r * dt) - d) / (u - d) # Risk-neutral probability of an up move

    # Initialize asset prices at maturity (N steps)
    # S_T[j] is the price if there are j up moves and N-j down moves
    S_T = np.zeros(N + 1)
    for j in range(N + 1):
        S_T[j] = S0 * (u**j) * (d**(N - j))

    # Initialize option values at maturity
    option_values = np.zeros(N + 1)
    if option_type == 'call':
        option_values = np.maximum(0, S_T - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - S_T)
    else:
        raise ValueError("Invalid option type. Must be 'call' or 'put'.")

    # Step backwards through the tree
    for i in range(N - 1, -1, -1): # From N-1 down to 0
        # At step i, there are i+1 possible option values
        new_option_values = np.zeros(i + 1)
        for j in range(i + 1): # j is the number of up moves up to step i
            # option_values[j] corresponds to the lower node after one more step
            # option_values[j+1] corresponds to the upper node after one more step
            new_option_values[j] = np.exp(-r * dt) * (q * option_values[j+1] + (1 - q) * option_values[j])
        option_values = new_option_values
        
    return option_values[0]

# --- 3. Monte Carlo Simulation ---
def monte_carlo_simulation(S0, K, T, r, sigma, M, option_type='call', random_seed=None):
    """
    Calculates the European option price using Monte Carlo simulation.

    Parameters:
    S0 (float): Current stock price
    K (float): Option strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate (annualized)
    sigma (float): Volatility of the underlying stock (annualized)
    M (int): Number of simulation paths
    option_type (str): 'call' or 'put'
    random_seed (int, optional): Seed for reproducibility

    Returns:
    float: Option price
    """
    if T == 0: # If time to expiration is 0, return intrinsic value
        if option_type == 'call':
            return max(0, S0 - K)
        elif option_type == 'put':
            return max(0, K - S0)
        else:
            raise ValueError("Invalid option type. Must be 'call' or 'put'.")

    if random_seed is not None:
        np.random.seed(random_seed)

    # Simulate stock prices at maturity T
    # S_T = S0 * exp((r - 0.5 * sigma^2) * T + sigma * sqrt(T) * Z)
    # where Z is a standard normal random variable
    
    Z = np.random.standard_normal(M) # M random numbers from standard normal distribution
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    # Calculate option payoffs at maturity
    if option_type == 'call':
        payoffs = np.maximum(S_T - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - S_T, 0)
    else:
        raise ValueError("Invalid option type. Must be 'call' or 'put'.")

    # Discount average payoff back to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

# --- Example Usage ---
if __name__ == "__main__":
    # Common parameters for a European Call Option
    S0 = 100      # Current stock price
    K = 105       # Strike price
    T = 1.0       # Time to expiration (1 year)
    r = 0.05      # Risk-free rate (5%)
    sigma = 0.2   # Volatility (20%)

    print(f"--- European Call Option ---")
    print(f"Parameters: S0={S0}, K={K}, T={T}, r={r}, sigma={sigma}\n")

    # Black-Scholes
    start_time = time.time()
    bs_call_price = black_scholes(S0, K, T, r, sigma, option_type='call')
    bs_time = time.time() - start_time
    print(f"Black-Scholes Call Price: {bs_call_price:.4f} (Time: {bs_time:.6f}s)")

    # Binomial Model
    N_binomial = 200  # Number of steps for Binomial model
    start_time = time.time()
    binomial_call_price = binomial_model_crr(S0, K, T, r, sigma, N_binomial, option_type='call')
    binomial_time = time.time() - start_time
    print(f"Binomial CRR Call Price (N={N_binomial}): {binomial_call_price:.4f} (Time: {binomial_time:.6f}s)")
    
    # For higher accuracy, N needs to be larger, but it becomes slower
    N_binomial_slow = 1000
    start_time = time.time()
    binomial_call_price_slow = binomial_model_crr(S0, K, T, r, sigma, N_binomial_slow, option_type='call')
    binomial_time_slow = time.time() - start_time
    print(f"Binomial CRR Call Price (N={N_binomial_slow}): {binomial_call_price_slow:.4f} (Time: {binomial_time_slow:.6f}s)")


    # Monte Carlo Simulation
    M_monte_carlo = 100000  # Number of simulations
    start_time = time.time()
    mc_call_price = monte_carlo_simulation(S0, K, T, r, sigma, M_monte_carlo, option_type='call', random_seed=42)
    mc_time = time.time() - start_time
    print(f"Monte Carlo Call Price (M={M_monte_carlo}): {mc_call_price:.4f} (Time: {mc_time:.6f}s)")
    
    M_monte_carlo_more = 1000000
    start_time = time.time()
    mc_call_price_more = monte_carlo_simulation(S0, K, T, r, sigma, M_monte_carlo_more, option_type='call', random_seed=42)
    mc_time_more = time.time() - start_time
    print(f"Monte Carlo Call Price (M={M_monte_carlo_more}): {mc_call_price_more:.4f} (Time: {mc_time_more:.6f}s)")


    print("\n--- European Put Option ---")
    # Common parameters for a European Put Option
    S0_put = 100
    K_put = 95
    T_put = 0.5
    r_put = 0.03
    sigma_put = 0.25
    print(f"Parameters: S0={S0_put}, K={K_put}, T={T_put}, r={r_put}, sigma={sigma_put}\n")

    bs_put_price = black_scholes(S0_put, K_put, T_put, r_put, sigma_put, option_type='put')
    print(f"Black-Scholes Put Price: {bs_put_price:.4f}")

    binomial_put_price = binomial_model_crr(S0_put, K_put, T_put, r_put, sigma_put, N_binomial, option_type='put')
    print(f"Binomial CRR Put Price (N={N_binomial}): {binomial_put_price:.4f}")

    mc_put_price = monte_carlo_simulation(S0_put, K_put, T_put, r_put, sigma_put, M_monte_carlo, option_type='put', random_seed=42)
    print(f"Monte Carlo Put Price (M={M_monte_carlo}): {mc_put_price:.4f}")

    print("\n--- Notes ---")
    print("1. Black-Scholes is analytical and fast for European options.")
    print("2. Binomial Model converges to Black-Scholes as N increases. Can be adapted for American options (by checking for early exercise at each node - not implemented here).")
    print("3. Monte Carlo converges as M (number of simulations) increases. Good for path-dependent or complex options. Standard MC is for European options; American options require more advanced MC techniques (e.g., Longstaff-Schwartz).")
    print("4. For Binomial, ensure risk-neutral probability 'q' is between 0 and 1. This means exp(r*dt) must be between d and u. This condition is generally met for CRR unless sigma is extremely low or dt is very large.")
    print("5. Time to expiration T=0 is handled as an edge case, returning intrinsic value.")

--- European Call Option ---
Parameters: S0=100, K=105, T=1.0, r=0.05, sigma=0.2

Black-Scholes Call Price: 8.0214 (Time: 0.001078s)
Binomial CRR Call Price (N=200): 8.0260 (Time: 0.169056s)
Binomial CRR Call Price (N=1000): 8.0211 (Time: 3.197137s)
Monte Carlo Call Price (M=100000): 8.0416 (Time: 0.013756s)
Monte Carlo Call Price (M=1000000): 8.0072 (Time: 0.109625s)

--- European Put Option ---
Parameters: S0=100, K=95, T=0.5, r=0.03, sigma=0.25

Black-Scholes Put Price: 4.0825
Binomial CRR Put Price (N=200): 4.0775
Monte Carlo Put Price (M=100000): 4.0837

--- Notes ---
1. Black-Scholes is analytical and fast for European options.
2. Binomial Model converges to Black-Scholes as N increases. Can be adapted for American options (by checking for early exercise at each node - not implemented here).
3. Monte Carlo converges as M (number of simulations) increases. Good for path-dependent or complex options. Standard MC is for European options; American options require more advanced MC tec